## Model: Model_Meta-Llama-3-8B-Instruct

In [12]:
!pip install -qU transformers accelerate sentence-transformers langchain pinecone-client datasets --quiet
!pip install -qU bitsandbytes --quiet
!pip install langchain spacy nltk --quiet
!pip install -U langchain-community --quiet
!pip install -U pinecone --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 20.0 MB/s eta 0:00:00


In [13]:
import os
from langchain_community.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema.document import Document
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import pandas as pd

# Load .env manually or define here
from google.colab import userdata
PINECONE_API_KEY=userdata.get('PINECONE_API_KEY')

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
df = pd.read_csv("processed_cleaned_chunks.csv")

docs = [
    Document(
        page_content=row['text'],
        metadata={"chunk_id": row['chunk_id'], "video_id": row['video_id']}
    )
    for _, row in df.iterrows()
]
df.to_csv('/content/drive/MyDrive/my_project/processed_cleaned_chunks.csv', index=False)

In [19]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'youtube-transcripts'
if index_name not in [i['name'] for i in pinecone.list_indexes()]:
    pinecone.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

vectors = []
for doc in docs:
    embedding = embed_model.encode(doc.page_content).tolist()
    vectors.append({
        'id': doc.metadata['chunk_id'],
        'values': embedding,
        'metadata': doc.metadata
    })

index = pinecone.Index(index_name)
index.upsert(vectors)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'upserted_count': 328}

In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True,
    token="YOUR_HF_TOKEN"
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:934: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct.
403 Client Error. (Request ID: Root=1-685da05a-06308c872bf6cbb6123ab2b1;724eba59-f4d8-4619-8138-5ab821c2a7a3)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted and you are not in the authorized list. Visit https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct to ask for access.